In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
import re
import mechanicalsoup
import pandas as pd
import webbrowser
from selenium import webdriver
import time
import random
import mechanicalsoup
import json

### 네이버 시리즈 장르별 일간 top 100 크롤링

In [76]:
naver_novel = pd.DataFrame()
label = ['title','genre','url','content','review_count']
genre_num = ['201','202','207','208','206','203','205','209']
for genre_id in genre_num:
    for i in range(0,6):
        url = f'https://series.naver.com/novel/top100List.series?rankingTypeCode=DAILY&categoryCode={genre_id}&page={i}'
        try:
            naver_headers = {}
            print(url, i)
            html = requests.get(url, headers = naver_headers, cookies = {}).text
            soup = BeautifulSoup(html, 'html.parser')
            product_count = len(soup.select('ul.comic_top_lst li'))
            for p in range(0,product_count):
                title = soup.select('ul.comic_top_lst li')[p].find_all('a')[1].get_text()
                print(title)
                detail_url = str("https://series.naver.com/")+soup.select('ul.comic_top_lst li')[p].find('a')['href']
                detail_html = requests.get(detail_url, headers=naver_headers, cookies={}).text
                #x = random.uniform(3, 5)
                #time.sleep(x)
                print('세부페이지: ', detail_url)
                body = BeautifulSoup(detail_html, 'html.parser')
                #            body = BeautifulSoup(driver.page_source, 'html.parser')
                browser = mechanicalsoup.StatefulBrowser()
                #더보기가 있으면 len = 2 없으면 len = 1
                #re.sub('\n','',url_soup.select('div._synopsis')[1].text)
                try :
                    a = body.select('div._synopsis')[1].text
                except:
                    a = body.select('div._synopsis')[0].text
                #desc에 쓰여있는 정규식 제거하기
                table = str.maketrans('\t\n\r\xa0\s+','       ')
                content = a.translate(table)
                #장르
                genre = body.select('li.info_lst li')[1].text
                #리뷰 관련
                bodyString = str(body)
                jsonStr = '{'+bodyString[bodyString.find("ghtProductInfo"):].split('}')[0].split('{')[1]+'}'
                jsonStr = jsonStr.replace("'",'"')
                dt = json.loads(jsonStr)
                origin_id = int(dt['originalProductId'])
                review_url = f'https://series.naver.com/comments/count.json?ticket=series5&objectId={origin_id}_0&lang=ko'
                review_html = requests.get(review_url, headers = naver_headers, cookies = {}).text
                review_count = json.loads(review_html, encoding="utf-8")['result']['comment']
                rx = [title,genre, detail_url, content,review_count]
                naver_novel = naver_novel.append(pd.Series(rx, index = label), ignore_index = True)
                x = random.uniform(0.5, 2.5)
                time.sleep(x)
        except Exception as e:
            print(e)
            continue
    #             if 'list index out of range' in str(e):
    #                 continue
    #             else:
    #                 time.sleep(3600)

https://series.naver.com/novel/top100List.series?rankingTypeCode=DAILY&categoryCode=201&page=0 0
https://series.naver.com/novel/top100List.series?rankingTypeCode=DAILY&categoryCode=201&page=1 1
만져도 돼? [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6548790


<ipython-input-76-857c271e59d2>:43: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  review_count = json.loads(review_html, encoding="utf-8")['result']['comment']


오빠집이 비어서 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=4983641
동창생 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6609739
매화가 흐드러지는 그 사이 [선공개]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6638245
못된 밤 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6537361
부부의 시간 [외전 포함][독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6294855
아마도, 굿모닝 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6135835
또다시, 계약 부부 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5692457
내 남편과 결혼해줘 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=4812845
내 안의 악마를 위하여 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=4855003
필요하다면, 사랑도 주고
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6642757
구원 후 애(愛) [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6646910
날 닮은 아이 [독점]
세부페이지:  https://series.naver.

사랑이 나를 부를 때 [외전 추가][독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6340925
새빨간 유혹 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5887787
관계의 방향 [선공개]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6494534
https://series.naver.com/novel/top100List.series?rankingTypeCode=DAILY&categoryCode=202&page=0 0
https://series.naver.com/novel/top100List.series?rankingTypeCode=DAILY&categoryCode=202&page=1 1
더 해머 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6484517
나 혼자 만렙 뉴비 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5206283
내가 키운 S급들
세부페이지:  https://series.naver.com//novel/detail.series?productNo=3777351
역대급 영지 설계사 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=4200133
나태 공자, 노력 천재 되다 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5769321
천마는 평범하게 살 수 없다 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5530937
검은머리 미군 대원수
세부페

죽지 않는 왕 - 무왕 단종
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5253898
고인물 신입생이 되었다 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6246814
만렙 잡캐
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5046786
위대한 명가의 막내아들이 되었다 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6507890
지구가 파산했다 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=4895629
말단 병사에서 군주까지
세부페이지:  https://series.naver.com//novel/detail.series?productNo=4064464
달빛조각사
세부페이지:  https://series.naver.com//novel/detail.series?productNo=1315886
아포칼립스 속 나만의 낙원
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6591845
개혁 군주 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6362478
회귀한 공작가의 막내도련님은 암살자 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6242687
삼국지 여포전
세부페이지:  https://series.naver.com//novel/detail.series?productNo=2181230
마법사의 검술이 너무 강함
세부페이지:  https://series.naver.com//nove

흑막 여주가 날 새엄마로 만들려고 해 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=4355065
애첩은 황성을 나가고 싶다 [선공개]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6537066
Lv.99 흑염의 프린세스 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6358880
사기 친 공작님을 유혹해버렸다 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5779613
악역의 엔딩은 죽음뿐
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5039321
원작에 없는 인물로 태어났습니다 [선공개]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6420347
악역 때려치우고 공방을 열었습니다 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6402433
공주님을 구원하겠습니다 [선공개]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5328662
그 결혼, 이번 생엔 제가 할게요 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6072781
https://series.naver.com/novel/top100List.series?rankingTypeCode=DAILY&categoryCode=207&page=5 5
악마는 벗어날 수 없다 [독점]
세부페이지:  https://series.naver.com//novel/detai

우주에서 온 독재자
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6628448
DO엔터 대표 도지훈 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5082721
너의 소울푸드가 보여!
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5297372
돈 쓸어담는 재벌가 서자
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6607327
아카데미의 재능만렙 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6321815
천재작가의 랜덤 작업실
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6342180
득템왕 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6404604
알고 보니 재벌 3세
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6372553
연극성 회귀 생활 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6469552
천재 사진작가 회귀하다 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6517206
삼도천 택시타고 힐링할래요
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6646935
신화급 귀속 아이템을 손에 넣었다 [독점]
세부페이지:  https://series.naver.co

무한 레벨업 in 무림 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=2500884
사상최강 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=2428694
https://series.naver.com/novel/top100List.series?rankingTypeCode=DAILY&categoryCode=206&page=4 4
천하제일 망나니 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6041800
일홀도 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5965388
신마 환생
세부페이지:  https://series.naver.com//novel/detail.series?productNo=5964949
모산기협전 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6144513
북검전기
세부페이지:  https://series.naver.com//novel/detail.series?productNo=1661216
나노 마신(喇勞 魔神) [단행본]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=3341382
17호 천재서생
세부페이지:  https://series.naver.com//novel/detail.series?productNo=6453642
사신빙의 [독점]
세부페이지:  https://series.naver.com//novel/detail.series?productNo=4920552
화산권마
세부페이지:  https://series.naver.com//novel/detail.series?productNo=331

### 네이버 웹툰 1570페이지까지 크롤링

In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
import re
import mechanicalsoup
import pandas as pd
import webbrowser
from selenium import webdriver
import time
import random
import json
import csv
# title, 마지막 연재일, 세부 url 가져오기
# page는 for문/ code = order
# 200page까지 돌리기
# label = ['title','genre','detail_url','date','review','content']
# f = open('naver_novel_tot.csv', 'a+', encoding = 'utf-8', newline='') # make new file if file doesn't exist

# wr = csv.writer(f)
# wr.writerow([label[0], label[1], label[2], label[3], label[4], label[5]])
f.close()
naver_novel = pd.DataFrame()
options = webdriver.ChromeOptions()
#agent가 없으면 bot으로 생각함
#options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
#driver = webdriver.Chrome(executable_path = './chromedriver',options=options)
#650~850
for page in range(640,650):
    print("페이지번호: ", page)
    # 페이지별 화면 켜기
    list_url = f'https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=all&genreCode=&orderTypeCode=new&is&isFinished=false&page={page}'
    naver_headers = {}
    html = requests.get(list_url, headers=naver_headers, cookies={}).text
    soup = BeautifulSoup(html, 'html.parser')
    browser = mechanicalsoup.StatefulBrowser()
    product_count = len(soup.select('ul.lst_list div'))
    for i in range(0, product_count):
        try:
            rx = []
            k = soup.select('ul.lst_list div')[i]
            title = k.find('a')['title']
            #print('제목: ', title)
            #날짜는 첫번째에 들어온다!
            tt = k.select('p.info')[0]
            date = tt.text.replace('\n','').split('|')[2]
            detail_url = str("https://series.naver.com/") + k.find('a')['href']
            #세부 정보 url 열기
    #             driver = webdriver.Chrome("./chromedriver")
    #             driver.get(detail_url)
            #print("세부화면 켜기: ", detail_url)
            detail_html = requests.get(detail_url,headers=naver_headers, cookies={}).text
            body = BeautifulSoup(detail_html, 'html.parser')
    #            body = BeautifulSoup(driver.page_source, 'html.parser')
            browser = mechanicalsoup.StatefulBrowser()
            #리뷰 관련
            bodyString = str(body)
            jsonStr = '{'+bodyString[bodyString.find("ghtProductInfo"):].split('}')[0].split('{')[1]+'}'
            jsonStr = jsonStr.replace("'",'"')
            dt = json.loads(jsonStr)
            origin_id = int(dt['originalProductId'])
            review_url = f'https://series.naver.com/comments/count.json?ticket=series5&objectId={origin_id}_0&lang=ko'
            review_html = requests.get(review_url, headers = naver_headers, cookies = {}).text
            review = json.loads(review_html, encoding="utf-8")['result']['comment']
            #print(review)
            #더보기가 있으면 len = 2 없으면 len = 1
            #re.sub('\n','',url_soup.select('div._synopsis')[1].text)
            try :
                a = body.select('div._synopsis')[1].text
            except:
                a = body.select('div._synopsis')[0].text
            #desc에 쓰여있는 정규식 제거하기
            table = str.maketrans('\t\n\r\xa0\s+','       ')
            content = a.translate(table)
            #장르
            genre = body.select('li.info_lst li')[1].text
    #            driver.close()
            rx = [title,genre, detail_url, date.replace('\t',''), review, content]
            naver_novel = naver_novel.append(pd.Series(rx, index = label), ignore_index = True)
            f = open(f'naver_novel_tot.csv', 'a+', encoding = 'utf-8', newline='')
            wr = csv.writer(f)
            wr.writerow(rx)
            f.close()
        except Exception as e:
            print(e)
            if 'list index out of range' in str(e):
                continue
            else:
                time.sleep(36)
naver_novel = naver_novel[['title','genre','date','url','review','content']]

페이지번호:  640


<ipython-input-5-e23c7c74eec7>:64: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  review = json.loads(review_html, encoding="utf-8")['result']['comment']


list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
페이지번호:  641
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
페이지번호:  642
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
페이지번호:  643
list index out of range
list index out of range
페이지번호:  644
list index out of range
list index out of range
list index out of range
list index out of range
페이지번호:  645
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list

KeyError: "['url'] not in index"

### 네이버시리즈 크롤링

In [181]:
naver_novel = pd.DataFrame()
weekdays = ['MON','TUE','WED','THU','TRI','SAT','SUN']
label = ['title','detail_url','download','content','genre','like']
for week in weekdays:    
    list_url = f'https://novel.naver.com/webnovel/weekdayList?week={week}&genre=all&order=Read'
    naver_headers = {}
    html = requests.get(list_url, headers=naver_headers, cookies={}).text
    soup = BeautifulSoup(html, 'html.parser')
    browser = mechanicalsoup.StatefulBrowser()
    product_count = len(soup.select('ul.list_type1 li'))
    print('작품개수: ',product_count)
    for i in range(0,product_count):
        rx = []
        body = soup.select('ul.list_type1 li')[i]
        title = body.find('a')['title']
        print('제목: ', title)
        detail_url = str("https://novel.naver.com") + body.find('a')['href']
        print(detail_url)
        detail_html = requests.get(detail_url, headers=naver_headers, cookies={}).text
        detail_soup = BeautifulSoup(detail_html, 'html.parser')
        browser = mechanicalsoup.StatefulBrowser()
        download = detail_soup.select('div.section_area_info span.download')[0].get_text()
        content = detail_soup.select('div.section_area_info p.dsc')[0].get_text()
        genre = detail_soup.select('div.section_area_info span.genre')[0].get_text()
        like = detail_soup.select('div.section_area_info span.like')[0].get_text()
        rx = [title,detail_url,download,content,genre,like]
        naver_novel = naver_novel.append(pd.Series(rx, index = label), ignore_index = True)
        x = random.uniform(0.5, 2.5)
        time.sleep(x)

작품개수:  40
제목:  재혼 황후
https://novel.naver.com/webnovel/list?novelId=765207
제목:  오빠는 왜 안 돼요?
https://novel.naver.com/webnovel/list?novelId=927907
제목:  공녀님의 꽃밭에는 그들이 산다
https://novel.naver.com/webnovel/list?novelId=920930
제목:  입술이 예쁜 남자
https://novel.naver.com/webnovel/list?novelId=949694
제목:  꽃이 삼킨 짐승
https://novel.naver.com/webnovel/list?novelId=994957
제목:  역대급 영지 설계사
https://novel.naver.com/webnovel/list?novelId=804489
제목:  죽음을 희망합니다
https://novel.naver.com/webnovel/list?novelId=963519
제목:  미친 중독
https://novel.naver.com/webnovel/list?novelId=994958
제목:  중매쟁이 아가 황녀님
https://novel.naver.com/webnovel/list?novelId=893642
제목:  레지나레나 - 용서받지 못한 그대에게
https://novel.naver.com/webnovel/list?novelId=868557
제목:  이별 후 사내 결혼
https://novel.naver.com/webnovel/list?novelId=933812
제목:  용두사망 소설 속의 악녀가 되었다
https://novel.naver.com/webnovel/list?novelId=933815
제목:  얘 우리 딸 아니에요!
https://novel.naver.com/webnovel/list?novelId=933818
제목:  간택주의보
https://novel.naver.com/webnovel/list?novelId=920925
제목:  더 캐슬
https

제목:  신과 함께 요리천재
https://novel.naver.com/webnovel/list?novelId=1002534
제목:  본업천재 톱매니저
https://novel.naver.com/webnovel/list?novelId=987044
제목:  치트 쓰는 아이돌
https://novel.naver.com/webnovel/list?novelId=1012540
제목:  부재의 시간
https://novel.naver.com/webnovel/list?novelId=949760
제목:  에이스 히어로
https://novel.naver.com/webnovel/list?novelId=994965
제목:  커피 뽑고 레벨 업!
https://novel.naver.com/webnovel/list?novelId=1012541
제목:  좀비묵시록 82-08
https://novel.naver.com/webnovel/list?novelId=1002528
제목:  던전 씹어먹는 아티팩트
https://novel.naver.com/webnovel/list?novelId=1012545
작품개수:  39
제목:  또다시, 계약 부부
https://novel.naver.com/webnovel/list?novelId=933800
제목:  끊을 수 없는 부부 사이
https://novel.naver.com/webnovel/list?novelId=963513
제목:  공녀님의 꽃밭에는 그들이 산다
https://novel.naver.com/webnovel/list?novelId=920930
제목:  폭군 남편과 이혼하겠습니다
https://novel.naver.com/webnovel/list?novelId=941327
제목:  부부, 어디까지 가능해?
https://novel.naver.com/webnovel/list?novelId=970355
제목:  新 광해의 연인
https://novel.naver.com/webnovel/list?novelId=814526
제목:  죽음을 희

제목:  그대에게 복수를 허락하노라
https://novel.naver.com/webnovel/list?novelId=1012530
제목:  천하제일 타이쿤
https://novel.naver.com/webnovel/list?novelId=987045
제목:  끈
https://novel.naver.com/webnovel/list?novelId=927908
제목:  이세계 강셰프
https://novel.naver.com/webnovel/list?novelId=957405
제목:  지구식 구원자 전형
https://novel.naver.com/webnovel/list?novelId=949762
제목:  나 홀로 지배자
https://novel.naver.com/webnovel/list?novelId=970358
제목:  치트 쓰는 아이돌
https://novel.naver.com/webnovel/list?novelId=1012540
제목:  커피 뽑고 레벨 업!
https://novel.naver.com/webnovel/list?novelId=1012541
제목:  좀비묵시록 82-08
https://novel.naver.com/webnovel/list?novelId=1002528
제목:  던전 씹어먹는 아티팩트
https://novel.naver.com/webnovel/list?novelId=1012545
작품개수:  40
제목:  하렘의 남자들
https://novel.naver.com/webnovel/list?novelId=853016
제목:  비서 말고 여자
https://novel.naver.com/webnovel/list?novelId=941322
제목:  또다시, 계약 부부
https://novel.naver.com/webnovel/list?novelId=933800
제목:  끊을 수 없는 부부 사이
https://novel.naver.com/webnovel/list?novelId=963513
제목:  폭군 남편과 이혼하겠습니다
https://no

### 카카오페이지 top 300 크롤링

In [38]:
label = ['title','read_count','date','genre','content','like_count','review']

# 무한 스크롤
# SCROLL_PAUSE_TIME = 2
# last_height = driver.execute_script("return document.body.scrollHeight")  
# while True:
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(SCROLL_PAUSE_TIME)                                     
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);") 
#     time.sleep(SCROLL_PAUSE_TIME)

# # Calculate new scroll height and compare with last scroll height           
#     new_height = driver.execute_script("return document.body.scrollHeight")

#     if new_height == last_height:                                               
#         break

#     last_height = new_height
kakao_novel = pd.DataFrame()
page = 1

category = ['86','120','89','117','87']
for category_num in category:
    print("장르: ",category_num)
    for page in range(0,12):
        time.sleep(1.5)
        print('페이지수:',page)
        url = f'https://api2-page.kakao.com/api/v1/store/today_rank/list?page={page}&category=11&subcategory={category_num}'
        driver = webdriver.Chrome("./chromedriver")
        driver.get(url)
        body = BeautifulSoup(driver.page_source, 'html.parser').get_text()
        body = json.loads(body, encoding="utf-8")
        driver.close()
        product_count = len(body['list'])
        for i in range(0, product_count):
            rx = []
            kakao_detail = body['list'][i]
            title = kakao_detail['title']
            print('제목: ',title)
#            caption = kakao_detail['caption']
            read_count = kakao_detail['read_count']
            date = kakao_detail['last_slide_added_date']
            genre = kakao_detail['sub_category_title']
            #kakao_detail['publisher']
            content = kakao_detail['description']
            table = str.maketrans('\t\n\r\xa0\s+','       ')
            content = content.translate(table)
            like_count = kakao_detail['like_count']
            review = kakao_detail['comment_count']
            #kakao_detail['like_count']
            rx = [title, read_count, date, genre, content, like_count, review]
            kakao_novel = kakao_novel.append(pd.Series(rx, index = label), ignore_index = True)
    
    
kakao_novel = kakao_novel[['title','read_count','date','genre','content','like_count','review']]
    

장르:  86
페이지수: 0


<ipython-input-38-73454b5cbffc>:32: DeprecationWarning: 'encoding' is ignored and deprecated. It will be removed in Python 3.9
  body = json.loads(body, encoding="utf-8")


제목:  템빨
제목:  이 용사 실화냐?
제목:  검술명가 막내아들
제목:  환생한 암살자는 검술 천재
제목:  천재 궁수의 스트리밍
제목:  2레벨로 회귀한 무신
제목:  네크로맨서 학교의 소환천재
제목:  얼어붙은 플레이어의 귀환
제목:  아카데미 마법천재로 사는 법
제목:  스킬빨로 레벨업
제목:  신과 함께 레벨업
제목:  암흑명가 천재 망나니가 되었다
제목:  나는 될놈이다
제목:  8클래스 마법사의 회귀
제목:  마탄의 사수
제목:  밥만 먹고 레벨업
제목:  해골병사는 던전을 지키지 못했다
제목:  뽑기로 강해진 10성급 플레이어
제목:  뽑기로 강해진 SSS급 헌터
제목:  SSS급 죽어야 사는 헌터
제목:  디버프 마스터
제목:  지옥에서 돌아온 성좌님
제목:  회귀했더니 무공 천재
제목:  대마법사도 정령사는 처음입니다
제목:  회귀한 엑스트라가 천재가 됨
페이지수: 1
제목:  전생자
제목:  역대급 창기사의 회귀
제목:  이세계 소환사는 프로게이머
제목:  나는 회귀자가 아닙니다
제목:  4000년 만에 귀환한 대마도사
제목:  1000년 만에 귀환한 검신
제목:  두 번 사는 랭커
제목:  내 인벤토리가 이상하다
제목:  열일하는 과금 기사
제목:  거병공 진군가
제목:  오늘부터 플레이어
제목:  부서진 성좌의 회귀
제목:  갑질하는 영주님
제목:  백작가의 망나니가 되었다
제목:  입학은 괜히 해 가지고
제목:  검의 순례자
제목:  엑스트라가 너무 강함
제목:  마왕은 학원에 간다
제목:  용을 삼킨 마법사
제목:  무림에서 온 배달부
제목:  만 년 만에 귀환한 플레이어
제목:  나 혼자만 레벨업
제목:  너무 맛있어서 탈이야!
제목:  절대자도 아빠는 처음이라
제목:  고인물은 옷을 입지 않아
페이지수: 2
제목:  나 혼자 소드 마스터
제목:  마신의 창고를 털었습니다
제목:  1RM의 기간트 라이더
제목:  달빛조각사
제목:  최후의 거인족이 되었다
제목:  삼국지 여포전
제목:  마법학교 마법사로 살아가는 법
제목:  닳고

페이지수: 8
제목:  퇴마하는 톱스타
제목:  내 아이돌의 꽃길을 위해
제목:  요리신동 막내아들
제목:  건반 위의 지배자
제목:  낙하산이 그렇게 일을 잘한다고?
제목:  퇴물은 월드스타가 되기로 했다
제목:  아프리카의 독재자가 되었다
제목:  퍼펙트 써전
제목:  우리집 베란다는 별을 품는다
제목:  스타 메이커
제목:  재벌가 막둥이는 만능천재
제목:  변혁 1990 [완결]
제목:  이번 생은 부동산이다!
제목:  겜공역전 세계의 게임천재가 되었다
제목:  두 번 사는 매니저
제목:  역대급 만능 BJ
제목:  격투의 신 [완결]
제목:  절대반지를 주움! [단행본]
제목:  재능만렙 미드라이너
제목:  복수의 끝은 재벌
제목:  에뜨랑제(Etranger)
제목:  23시 59분
제목:  조선, 혁명의 시대 2부
제목:  스토퍼 [완결]
제목:  걸그룹 멤버의 남동생이 되었다 [연재]
페이지수: 9
제목:  신화의 전장
제목:  전신의 탄생 : 양아치
제목:  천재, 세상을 읽다 [완결]
제목:  레벨업 닥터 최기석 [완결]
제목:  천재 프로파일러 임한솔
제목:  하트 받는 탑스타
제목:  늑대전설
제목:  스트리머가 못하는 게 없음
제목:  귀환했는데 입대 전날이다
제목:  요리사가 축복을 숨김
제목:  카드빨로 축구 스타
제목:  초월자 학원의 수강생이 되었다 [완결]
제목:  우리 삼촌은 월드스타
제목:  리미트리스 드림
제목:  은퇴 좀 해라
제목:  과거에서 재능이 쏟아져
제목:  사람답게 살자
제목:  어딜 봐요 매니저님
제목:  버프받고 탑스타
제목:  티무르 사용 설명서
제목:  나 혼자 천재 재벌로 레벨업(케미 천마)
제목:  산지직송 자연산 천재배우 [완결]
제목:  더 프레지던트: 트레저헌터
제목:  졸부 집 망나니[2부 연재]
제목:  별이 되다[완결]
페이지수: 10
제목:  만능 유학생이 되었다 [연재]
제목:  생존학 개론
제목:  운명을 보는 회사원 [완결]
제목:  만능사원 전설이 되다
제목:  철수를 구하시오
제목

페이지수: 4
제목:  악역 황녀님은 과자집에서 살고 싶어
제목:  신데렐라를 곱게 키웠습니다 [완결]
제목:  아빠, 나 이 결혼 안 할래요!
제목:  신데렐라는 내가 아니었다
제목:  버림받고 즐기는 소박한 독신의 삶
제목:  흑막 남주인공 아이들의 엄마가 되었다
제목:  소공녀는 파혼을 기다린다
제목:  폭군의 애착인형
제목:  에보니
제목:  로판을 모르면 죽습니다
제목:  왜 이러세요, 시어머니?
제목:  장르를 바꿔보도록 하겠습니다
제목:  남주인 줄 모르고 선을 넘었다
제목:  황녀님이 사악하셔
제목:  두 분은 훗날, 저를 낳습니다
제목:  책빙의는 이제 지겹다
제목:  아가씨는 새 인생 찾을래!
제목:  내 소원은 침대 두 개
제목:  누군가 내 몸에 빙의했다
제목:  악당의 아빠를 꼬셔라
제목:  환생 공녀님
제목:  우선 남동생부터 숨기자
제목:  아 하세요, 금수저 들어갑니다
제목:  인소의 법칙
제목:  순백의 엘리사벳
페이지수: 5
제목:  황녀, 반역자를 각인시키다
제목:  핸드 메이드
제목:  남편이 빙의한 주인공이다
제목:  뱀 가문의 막내딸입니다
제목:  그 아빠에 그 딸
제목:  악역 소녀는 사랑받기가 부끄러워요
제목:  건강이 없습니다
제목:  남주를 꼬시려던 건 아니었습니다
제목:  피폐물이라 등장인물들이 심신미약이다
제목:  그는 내 여동생을 사랑했다
제목:  역하렘 게임 속으로 떨어진 모양입니다
제목:  북부 대공의 정체가 흑막 악녀라 곤란하다
제목:  우리가 정략결혼 하는 진짜 이유
제목:  주치의는 할 일 다 하고 사표 씁니다
제목:  레이디 에디타 [완결]
제목:  내 약혼자의 애인을 찾습니다 [특별외전 완결]
제목:  폭군의 유일한 조향사가 되었습니다
제목:  치트라 - 미남을 뽑는 여백작 [완결]
제목:  남주가 내 후원을 좋아해!
제목:  원작을 건드리면 생기는 일
제목:  가짜 성녀인데 신들이 집착한다
제목:  폐하, 또 죽이진 말아주세요
제목:  악당들을 돈으로 사 버렸습니다
제목:  서브 남주네 기사단장

### 정리

In [85]:
naver_novel_fin = naver_novel[['title','genre','content','review_count','url']]

In [86]:
naver_novel_fin['platform'] = '<h1>네이버</h1>'
naver_novel_fin = naver_novel_fin[['platform','title','genre','content','review_count']]

<ipython-input-86-6ecedaecd3d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_novel_fin['platform'] = '<h1>네이버</h1>'


In [87]:
kakao_novel['platform'] = '<h1>카카오</h1>'
kakao_novel_fin = kakao_novel[['platform','title','genre','content','review']]

In [89]:
naver_novel_fin.rename(columns = {"review_count": "review"}, inplace = True)

In [90]:
fin_novel_df = pd.concat([naver_novel_fin, kakao_novel_fin]) 

In [92]:
fin_novel_df.to_excel('popular_novel.xlsx',header=True)

In [74]:
naver_novel_fin['genre'].unique()

array(['로맨스', '판타지', '로판', '현판', '무협'], dtype=object)

In [75]:
kakao_novel_fin['genre'].unique()
#판타지 / 현판 / 로맨스 / 무협

array(['판타지소설', '현판소설', '로맨스소설', '로판소설', '무협소설'], dtype=object)